# Ejercicio OpenStreetMap
Para este ejercicio vas a trabar con Overpass, que es la API que utiliza los datos de OpenStreetMap. Tras obtener datos geográficos mediante la API, tendrás que representarlos con Plotly.

Hay que obtener los siguientes datos de cada ubicación conseguida en OpenStreetMap
* tipo de amenity
* latitud
* longitud
* nombre
* direccion

**NOTA**: En este Notebook utilizarás gráficas de Plotly, y JupyterLab de momento no lo soporta, a no ser que instales una extensión. Para solucionar esto, abre el Notebook desde Jupyter normal:

![imagen](./img/jupyter.png)

Tienes en [este enlace](https://wiki.openstreetmap.org/wiki/Key:amenity#Healthcare) la lista de *amenities* que vas a usar en el notebook.

## Instalación
Si no tienes instalado Plotly, abre un terminal y escribe:
```
pip install plotly
```

Importa las librerias que vamos a usar en el Notebook

In [1]:
import requests
import json
import pandas as pd
import plotly.express as px



Ataca a la API de OpenStreetmap con el siguiente código de ejemplo

In [8]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="ice_cream"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()

<Response [200]>


Con el siguiente código podrás adaptar el JSON que devuelve la API, a un formato DataFrame más amigable

In [7]:
def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)
    

Utilizarás esta función para representar los datos en un mapa de Matplotlib

In [19]:
import plotly.express as px

mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()
    


Como ejemplo, si quiero ver cuántos restaurantes tengo cerca de The Bridge:

In [9]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="restaurantes"]
  (40.420801, -3.694702,40.423754, -3.688167); 
[out:json]; 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [400]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
pinta_mapa(df)

## Ejercicio 1
### Búsqueda de aparcamiento
Para este caso de uso imagina que vas a Madrid de viaje y estás buscando posibles parkings donde dejar el coche, crea un DataFrame con todos los parkings (de OpenStreetMap) de la ciudad de Madrid. Para ello define una cuadrícula con dos coordenadas sobre Madrid.

Representa en un mapa los resultados

In [11]:
import requests
import json
import pandas as pd


In [21]:

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="parking"]
  (40.2387,-3.9647,40.5368, -3.2671);
out;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df







<Response [200]>


,tipo,lat,lon,name,address
0,parking,40.394324,-3.794812,NO NAME,NO STREET 9999
1,parking,40.394064,-3.794909,NO NAME,NO STREET 9999
2,parking,40.398227,-3.783833,NO NAME,NO STREET 9999
3,parking,40.426712,-3.714370,Princesa 25,NO STREET 9999
4,parking,40.414469,-3.703549,Parking Benavente,NO STREET 9999
...,...,...,...,...,...
542,parking,40.372956,-3.773751,NO NAME,NO STREET 9999
543,parking,40.395501,-3.794887,NO NAME,NO STREET 9999
544,parking,40.393807,-3.798228,NO NAME,NO STREET 9999
545,parking,40.320327,-3.756167,NO NAME,NO STREET 9999


In [22]:
import plotly.express as px

mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()


In [23]:
pinta_mapa(df)

## Mudanza
Imagina que te quieres mudar al Berrueco, pero antes te gustaría obtener algún indicador de si hay civilización por esa zona. Mira a ver si existe gasolineras y/o colegios por la zona. Define, al igual que en el caso anterior, un rectángulo mediante dos coordenadas. Busca la manera de realizar una única petición a la API.

Representa después en un mapa, por color los colegios y gasolineras

In [27]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"~"fuel|school"]
  (40.8637, -3.6071, 40.9105, -3.5136);
out;
"""

response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data1 = response.json()
df1 = json_to_df(data1)
df1


<Response [200]>


,tipo,lat,lon,name,address
0,music_school,40.872876,-3.606421,Escuela Municipal De Musica Y Danza La Cabrera,NO STREET 96
1,fuel,40.875833,-3.602389,Shell,Avenida de La Cabrera 9999


In [28]:
pinta_mapa(df1)

## Zonas COVID
Imagina que trabajas en el ayuntamiento de tu barrio/pueblo/ciudad, y por motivos del COVID tienes que identificar:
1. Todas las zonas abiertas de uso común (zonas de barbacoas, fuentes de agua y parques infantiles (kindergarten)), para cerrarlas.
2. Todos los hospitales y clínicas de la zona.

Como no sabes muy bien qué puntos serán, utilizas la API de OpenStreetMap. Representa todo en un único mapa

In [19]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"~"clinic|hospital|kindergarten|drinking_water|bbq"] 
 (37.3434, -6.0171, 37.4411, -5.9388);
out;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
print(response)
data4 = response.json()
df4 = json_to_df(data4)
df4


<Response [200]>


,tipo,lat,lon,name,address
0,drinking_water,37.387834,-6.001198,Fuente para beber agua potable,NO STREET 9999
1,drinking_water,37.393873,-5.983184,NO NAME,NO STREET 9999
2,drinking_water,37.393116,-5.986985,NO NAME,NO STREET 9999
3,drinking_water,37.392572,-5.989869,NO NAME,NO STREET 9999
4,drinking_water,37.398458,-5.981853,NO NAME,NO STREET 9999
...,...,...,...,...,...
280,clinic,37.374022,-5.994595,Área Salud Ocular - Consultas,NO STREET 9999
281,clinic,37.376331,-6.001857,Alfamedica Centro de Rehabilitación,Calle Pedro Pérez Fernández 7
282,clinic,37.388650,-6.011749,NO NAME,Calle Rafael Belmonte García 9999
283,drinking_water,37.385240,-5.998645,NO NAME,NO STREET 9999


In [21]:
mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()


In [22]:
pinta_mapa(df4)

## Supermercados
Para este ejemplo supondrás que trabajas en una cadena de supermercados en la que se está planteando construir un nuevo local en la provincia de Sevilla. Busca mediante la API de Overpass todos los supermercados de la provincia para poder comprobar en qué nucleos urbanos habría poca competencia.

En este caso hay que buscar por área, no por cuadrícula! Investiga en la documentación/Stackoverflow cómo se haría eso.

In [32]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["shop"="supermarket"] 
 (37.3434, -6.0171, 37.4411, -5.9388);
out;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
print(response)
data5 = response.json()
df5 = json_to_df(data5)
df5


<Response [200]>


,tipo,lat,lon,name,address
0,None,37.390016,-5.973769,Maxi Dia,NO STREET 9999
1,None,37.406364,-5.976342,El Jamón,NO STREET 9999
2,None,37.406508,-5.973342,Valdejimena,Avenida de San Juan de la Salle 9999
3,None,37.406889,-5.972455,Dia,Avenida de San Juan de la Salle 12
4,None,37.381381,-5.979418,MAS,NO STREET 9999
...,...,...,...,...,...
159,None,37.344104,-5.989356,Mercadona,NO STREET 9999
160,None,37.384955,-5.961366,CensoredShop,NO STREET 9999
161,None,37.393974,-5.992695,Carrefour Express,NO STREET 9999
162,None,37.376837,-5.974045,Dia,Avenida Ramón y Cajal 12


In [ ]:
def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()


In [33]:
pinta_mapa(df5)

## Etiqueta en el mapa
Busca la manera de modificar la función `pinta_mapa()`, de tal manera que cuando pases el ratón por encima de cada punto, aparezca el nombre de cada nodo. Por ejemplo, que en el último apartado aparezcan los nombres de los supermercados.

In [37]:
df_super = df5.copy()
df_super["tipo"] = df5["name"]
df_super

,tipo,lat,lon,name,address
0,Maxi Dia,37.390016,-5.973769,Maxi Dia,NO STREET 9999
1,El Jamón,37.406364,-5.976342,El Jamón,NO STREET 9999
2,Valdejimena,37.406508,-5.973342,Valdejimena,Avenida de San Juan de la Salle 9999
3,Dia,37.406889,-5.972455,Dia,Avenida de San Juan de la Salle 12
4,MAS,37.381381,-5.979418,MAS,NO STREET 9999
...,...,...,...,...,...
159,Mercadona,37.344104,-5.989356,Mercadona,NO STREET 9999
160,CensoredShop,37.384955,-5.961366,CensoredShop,NO STREET 9999
161,Carrefour Express,37.393974,-5.992695,Carrefour Express,NO STREET 9999
162,Dia,37.376837,-5.974045,Dia,Avenida Ramón y Cajal 12


In [38]:
pinta_mapa(df_super)
